<a href="https://colab.research.google.com/github/luasm17/LLM_as_a_judge/blob/main/prueba_1_prometheus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instalar dependencias + Imports
!pip install -q -U "transformers>=4.51.0" accelerate safetensors

import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 21.5 MB/s eta 0:00:00


In [2]:
# Token HF
HF_TOKEN = os.environ.get("HF_TOKEN", None)

In [3]:
# Cargar modelo e tokenizer (M-Prometheus-7B)
# LLM-as-a-Judge binario para concordancia de número en galego

MODEL_ID = "Unbabel/M-Prometheus-7B"

print("🔄 Cargando tokenizer...")
if HF_TOKEN:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, token=HF_TOKEN)
else:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

print("🔄 Cargando modelo (pode tardar uns minutos)...")
if HF_TOKEN:
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        token=HF_TOKEN,
        device_map="auto",
        torch_dtype="auto"
    )
else:
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        device_map="auto",
        torch_dtype="auto"
    )

model.eval()
print("✅ Modelo cargado correctamente")

🔄 Cargando tokenizer...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json:   0%|          | 0.00/790 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/339 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

✅ Modelo cargado correctamente


In [4]:
# Función LLM-as-a-Judge (concordancia de número)
def prometheus_judge_concordancia(oracion: str) -> str:
    prompt = f"""
Es un LLM-as-a-judge que está a avaliar a saída dun modelo de corrección gramatical (GEC) en galego.

A túa tarefa: decidir se a frase dada contén un erro de CONCORDANCIA DE NÚMERO
(concordancia singular/plural entre determinante, substantivo, adxectivo ou verbo).

DEBES avaliar EXCLUSIVAMENTE a concordancia de número. NON avalíes ningún outro tipo de erro.
NON DEBES, baixo ningún concepto, corrixir nin reescribir a frase. Limítate unicamente a xulgala.

Rúbrica (binaria):
- [RESULT] 1 -> A corrección NON é adecuada (segue existindo un erro de concordancia de número)
- [RESULT] 0 -> A corrección é adecuada (NON hai erro de concordancia de número)

Formato de saída (ESTRICTO, sen texto adicional):
Feedback: "<xustificación breve e precisa en galego centrada EXCLUSIVAMENTE na concordancia de número>"
[RESULT] <0 ou 1>

Agora avalía a seguinte saída dun modelo de GEC:

<OUTPUT_MODELO>
{oracion}
</OUTPUT_MODELO>
"""

    messages = [{"role": "user", "content": prompt}]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer(text, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.0,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )

    resposta = tokenizer.decode(
        output[0][inputs["input_ids"].shape[-1]:],
        skip_special_tokens=True
    )

    return resposta.strip()

In [5]:
# Probas cos exemplos
exemplos = [
    "As decisións tomadas polo comité foron comunicadas aos responsables das distintas áreas.",
    "O grupos de estudantes que participou no proxecto presentou os resultados finais onte pola tarde.",
    "As propostas que chegaron desde os concellos máis pequenos foi analizadas polo equipo técnico.",
    "A maioría das persoas entrevistadas manifestou a súa opinión durante a sesión pública.",
    "Os relatorios anuais sobre o impacto ambiental foi revisados exhaustivamente pola comisión de avaliación.",
    "A valiosa colección de poemas inéditos do autor galego publicouse recentemente baixo un prestixioso selo independente.",
    "O paquete de medidas económicas recentemente aprobado é de aplicación inmediata en todos os sectores da economía.",
    "As adversas condicións climáticas dos últimos días obrigou a suspender completamente varias actividades programadas ao aire libre.",
    "A maioría dos participantes no curso de formación mostrou un grande interese en continuar con sesións prácticas adicionais.",
    "Os sabios consellos que me deches sobre a xestión do tempo foi moi útiles para poder tomar unha decisión acertada.",
]

for i, frase in enumerate(exemplos, 1):
    print(f"\n===== EXEMPLO {i} =====")
    print(prometheus_judge_concordancia(frase))

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



===== EXEMPLO 1 =====
Feedback: "O verbo 'foron' está en concordancia incorrecta coa subestantivo 'decisións', que é plural. O verbo debe estar en singular para concordar con o subestantivo singular 'decisión'." 
[RESULT] 1

===== EXEMPLO 2 =====
Feedback: "O erro está na concordancia do verbo 'presentou' que debe estar en plural 'presentaron' para concordar coa subestantivo 'grupos'"
[RESULT] 1

===== EXEMPLO 3 =====
Feedback: "O verbo 'foi' está en singular, pero o sujeito 'propostas' é plural, causando un erro de concordancia de número"
[RESULT] 1

===== EXEMPLO 4 =====
Feedback: "O verbo 'manifestou' está en singular, pero o sujeto 'a maioría das persoas' é plural, causando un erro de concordancia de número"
[RESULT] 1

===== EXEMPLO 5 =====
Feedback: "O erro de concordancia de número está presente no verbo 'foi', que debe estar en plural para concordar coa subestantivo 'relatorios' e 'impacto'. O verbo debe ser 'foron' para que concorde adequadamente."
[RESULT] 1

===== EXEMPLO 6